# Tutorial 2: Working with Language Models in LangChain

In this tutorial, we'll explore how to work with language models in LangChain, focusing on the Groq LLM. We'll cover connecting to the model, creating prompt templates, building chains, and handling responses.

## 1. Connecting to Language Models

First, let's set up our environment and connect to the Groq LLM:

In [10]:
import os
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv

load_dotenv()

# Initialize the Groq LLM
llm = ChatGroq(
        model_name="llama-3.1-70b-versatile",
        temperature=0.1,
        model_kwargs={"top_p": 0.2, "seed": 1337}
    )

# Test the connection
response = llm.invoke("Hello, world!")
print(response)

content='Hello. How can I assist you today?' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 39, 'total_tokens': 49, 'completion_time': 0.04, 'prompt_time': 0.007011686, 'queue_time': 0.014202011, 'total_time': 0.047011686}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_9260b4bb2e', 'finish_reason': 'stop', 'logprobs': None} id='run-6adbb062-bbed-499a-97d7-15c574792650-0' usage_metadata={'input_tokens': 39, 'output_tokens': 10, 'total_tokens': 49}


## 2. Creating Prompt Templates

Prompt templates allow us to create reusable prompts with input variables:

In [11]:
# Define a simple prompt template
template = """Answer the following question:
You answers in the {language} language.
Question: {question}
Answer: Let's approach this step-by-step:"""


prompt = PromptTemplate(template=template, input_variables=["question","language"])

# Use the prompt template
question = "What is the capital of France?"
formatted_prompt = prompt.format(question=question,language="English")
print(formatted_prompt)

Answer the following question:
You answers in the English language.
Question: What is the capital of France?
Answer: Let's approach this step-by-step:


## 3. Building Simple Prompt Chains

Now, let's create a chain that combines our prompt template with the language model:

In [12]:
# Create a chain
chain = prompt | llm

# Run the chain
result = chain.invoke({"question":"What is the speed of light?","language":"French"})
print(result.content)

Approchons cela étape par étape :

La vitesse de la lumière est une constante physique fondamentale qui représente la vitesse à laquelle la lumière se propage dans le vide. Elle est notée c et est exprimée en mètres par seconde (m/s).

La vitesse de la lumière est de 299 792 458 mètres par seconde. Cette valeur a été déterminée avec une grande précision grâce à des expériences et des mesures précises.

Il est important de noter que la vitesse de la lumière est une constante universelle, ce qui signifie qu'elle est la même partout dans l'univers et qu'elle ne dépend pas de la vitesse de l'observateur ou de la source de lumière.


## 4. Handling Model Responses

Let's explore different ways to handle and process model responses:

In [13]:
# Get the raw response
raw_response = llm.invoke("List three prime numbers.")
print("Raw response:", raw_response)

# Using the chain with a dictionary input
chain_response = chain.invoke({"question": "tree Names","language": "French"})
print("\nChain response:", chain_response.content)

Raw response: content='Here are three prime numbers:\n\n1. 7\n2. 11\n3. 23' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 40, 'total_tokens': 61, 'completion_time': 0.084, 'prompt_time': 0.007700269, 'queue_time': 0.030701834, 'total_time': 0.091700269}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_b3ae7e594e', 'finish_reason': 'stop', 'logprobs': None} id='run-c6302169-d8f7-4d6d-ba1e-15b92ab3b158-0' usage_metadata={'input_tokens': 40, 'output_tokens': 21, 'total_tokens': 61}

Chain response: Pour commencer, voici quelques noms d'arbres courants en français :

1. Arbre fruitier : Pommier (Pomme), Pêcher (Pêche), Cerisier (Cerise)
2. Arbre à feuilles : Chêne, Hêtre, Érable
3. Arbre à aiguilles : Pin, Sapin, Épicéa
4. Arbre tropical : Palmier, Cocotier, Manglier
5. Arbre ornemental : Lilas, Fleur d'oranger, Magnolia

Voici quelques autres exemples :

* Le chêne est un arbre robuste et durable.
* Le pommier est un a

In [14]:
# Parsing structured output
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

list_prompt = PromptTemplate(
    template="List 100 {item}. {format_instructions} write only the colors, nothing else",
    input_variables=["item"],
    partial_variables={"format_instructions": format_instructions}
)

chain = list_prompt | llm |output_parser
result = chain.invoke({"item":"colors"})
print(type(result))
print("\nParsed list:", result)

<class 'list'>

Parsed list: ['Red', 'Orange', 'Yellow', 'Green', 'Blue', 'Indigo', 'Violet', 'Pink', 'Brown', 'Grey', 'Black', 'White', 'Turquoise', 'Silver', 'Gold', 'Copper', 'Bronze', 'Beige', 'Cream', 'Ivory', 'Lavender', 'Peach', 'Magenta', 'Cyan', 'Teal', 'Coral', 'Salmon', 'Maroon', 'Navy', 'Purple', 'Olive', 'Lime', 'Mint', 'Aqua', 'Fuchsia', 'Charcoal', 'Taupe', 'Mocha', 'Sage', 'Seafoam', 'Periwinkle', 'Plum', 'Burgundy', 'Onyx', 'Ruby', 'Emerald', 'Garnet', 'Amethyst', 'Jade', 'Cobalt', 'Cerulean', 'Azure', 'Sapphire', 'Tangerine', 'Lemon', 'Powder Blue', 'Rose', 'Blush', 'Mauve', 'Sienna', 'Umber', 'Terracotta', 'Carmine', 'Crimson', 'Scarlet', 'Vermilion', 'Coral Red', 'Russet', 'Clay', 'Sand', 'Khaki', 'Tan', 'Caramel', 'Honey', 'Mustard', 'Sienna Brown', 'Mahogany', 'Walnut', 'Ebony', 'Ivory Black', 'Pearl', 'Opal', 'Amaranth', 'Caput Mortuum', 'Carmine Red', 'Celadon', 'Cerise', 'Cinnabar', 'Cochineal', 'Gamboge', 'Heliotrope', 'Smalt', "Scheele's Green", 'Rose Madder'

## 5. Best Practices for Prompt Engineering

Here are some best practices for effective prompt engineering:

In [15]:
# 1. Be specific and provide context
specific_prompt = PromptTemplate(
    template="You are an expert in {field}. Explain {concept} in simple terms for a beginner.",
    input_variables=["field", "concept"]
)

# 2. Use examples (few-shot learning)
few_shot_prompt = PromptTemplate(
    template="""Classify the sentiment of the following text as positive, negative, or neutral.

Example 1:
Text: I love this product!
Sentiment: Positive

Example 2:
Text: This is the worst experience ever.
Sentiment: Negative

Example 3:
Text: The weather is cloudy today.
Sentiment: Neutral

Now, classify the following text:
Text: {text}
Sentiment:""",
    input_variables=["text"]
)

# 3. Break complex tasks into steps
step_prompt = PromptTemplate(
    template="""To solve the problem '{problem}', follow these steps:
1. Identify the key information
2. Determine the appropriate formula or method
3. Apply the formula or method step-by-step
4. Check your answer

Now, solve the problem:""",
    input_variables=["problem"]
)

# Test the prompts
chains = {
    "Specific": specific_prompt | llm,
    "Few-shot": few_shot_prompt | llm,
    "Step-by-step": step_prompt | llm 
    }

for name, chain in chains.items():
    print(f"\n---------------------------------------------------\n{name} Prompt Result:")
    if name == "Specific":
        print(chain.invoke({"field":"physics", "concept":"quantum entanglement"}).content)
    elif name == "Few-shot":
        print(chain.invoke({"text":"This movie was okay, I guess."}).content)
    else:
        print(chain.invoke({"problem":"Calculate the area of a circle with radius 5 cm"}).content)


---------------------------------------------------
Specific Prompt Result:
Quantum entanglement is a fascinating concept in physics that can be a bit tricky to grasp, but I'll try to explain it in simple terms.

**What is Quantum Entanglement?**

Imagine you have two toy cars that are connected by a spring. If you push one car, the other car will move too, because they're connected by the spring. This is a classical example of how two objects can be connected and affect each other.

Now, imagine that these toy cars are not connected by a spring, but they're still somehow "linked" in a way that lets them affect each other, even if they're on opposite sides of the universe. This is roughly what quantum entanglement is.

**How Does it Work?**

In the quantum world, tiny particles like atoms and electrons can become "entangled" in a way that lets them affect each other, even if they're separated by huge distances. When two particles are entangled, their properties (like spin, energy, or 

## Conclusion

In this tutorial, we've explored various aspects of working with language models in LangChain, including connecting to models, creating prompt templates, building chains, handling responses, and implementing best practices for prompt engineering. These skills will serve as a foundation for building more complex applications with LangChain in future tutorials.